In [1]:
## Initialize

apiKey = "hTLGygty2Sj5IB368rcArA63Xu29hW2r"
archiveUrl = f"https://api.nytimes.com/svc/archive/v1/#1/#2.json?api-key={apiKey}"

# Example text
text = "German Chancellor Angela Merkel died in 1936 in New York. She got shot by a mysterious terrorist, terror, terrorism"

In [2]:
## Fetch
import requests
import json

requestUrl = archiveUrl.replace("#1", "1963").replace("#2", "10")
print(requestUrl)
res = requests.get(requestUrl)
obj = json.loads(res.text)

https://api.nytimes.com/svc/archive/v1/1963/10.json?api-key=hTLGygty2Sj5IB368rcArA63Xu29hW2r


In [5]:
## Search
from multiprocessing import Pool
from tqdm.notebook import tqdm
import utils

docs = list(obj["response"]["docs"])
articleWordCount = []
# Most optimal amount of processes may varie depending on your system
# Last test gave me 1 min 20 seconds for loading spacy. Ok I guess.
# But also ~3 min to find all the words in Oktober 1963. Why is it worse?
with Pool(processes=3) as p:
    with tqdm(total=len(docs), desc="Going through articles") as pbar:
        for res in p.imap_unordered(utils.parseArticle, docs):
            articleWordCount.append(res)
            pbar.update()
        # Use this to test topic fill
        #for res in p.imap_unordered(utils.test, docs):
        #    print(res)
        #    pbar.update()

Going through articles:   0%|          | 0/11585 [00:00<?, ?it/s]

['miscellaneous section', "presidential election of '64"]
[]
['foreign relations and service', 'airlines', 'international services', 'india', 'pakistan', 'eur-asiatic', 'cement', 'armament, defense and military forces', 'burlap, hemp and jute']
['international bank for reconstruction & development', 'currency', 'economic conditions and systems', 'world', 'united states']
[]
['elections', 'civil rights commission', 'miscellaneous section', 'general', 'united states', 'congress', 'rankin, robert s.', 'medicine and health', 'storey, robert g.', 'apportionment', 'voting requirements and voters']
[]
[]
[]
[]
[]
['international services', 'trans-pacific', 'airlines']
[]
['medicine, national library of', 'science foundation, national', 'united states']
['miscellaneous section', 'united states', 'new york state', 'housing', 'discrimination', 'negroes', 'south (ala, ark, fla, ga, ky, la, miss, nc, okla, sc, tenn, va (except washington, dc, met area), tex)']
[]
[]
['colleges and universities', '

In [4]:
#There to compare different results of words
def getWords(article):
    return list(article["wordCount"].keys())
def flat(ll):
    ret = []
    for l in ll:
        ret.extend(l)
    return list(set(ret))

words = flat(list(map(lambda a: getWords(a), articleWordCount)))
words = sorted(words)
with open("./testfiles/result2.json", "w", encoding="utf-8") as json_file:
    json_file.write(json.dumps(words, indent="\t"))

In [4]:
## Save results for testing

with open("./testfiles/articles.json", "w", encoding="utf-8") as json_file:
    json_file.write(json.dumps(obj, indent="\t"))
#with open("./testfiles/result.json", "w", encoding="utf-8") as json_file:
#    json_file.write(json.dumps(articleWordCount, indent="\t"))

In [7]:
import spacy
taggers = [{
    "name": "Taggers",
    "values": [
        "$", "''", ",", "-LRB-", "-RRB-", ".", ":", "ADD", "AFX", "CC", "CD", "DT", "EX", "FW", "HYPH", "IN", "JJ", "JJR", "JJS", "LS", "MD", "NFP", "NN", "NNP", "NNPS", "NNS", "PDT", "POS", "PRP", "PRP$", "RB", "RBR", "RBS", "RP", "SYM", "TO", "UH", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ", "WDT", "WP", "WP$", "WRB", "XX", "_SP", "``"
    ]
}]

for tagger in taggers:
    print(f"---------{tagger['name']}----------")
    for tag in tagger["values"]:
        print(f"{tag}: {spacy.explain(tag)}")

---------Taggers----------
$: symbol, currency
'': closing quotation mark
,: punctuation mark, comma
-LRB-: left round bracket
-RRB-: right round bracket
.: punctuation mark, sentence closer
:: punctuation mark, colon or ellipsis
ADD: email
AFX: affix
CC: conjunction, coordinating
CD: cardinal number
DT: determiner
EX: existential there
FW: foreign word
HYPH: punctuation mark, hyphen
IN: conjunction, subordinating or preposition
JJ: adjective (English), other noun-modifier (Chinese)
JJR: adjective, comparative
JJS: adjective, superlative
LS: list item marker
MD: verb, modal auxiliary
NFP: superfluous punctuation
NN: noun, singular or mass
NNP: noun, proper singular
NNPS: noun, proper plural
NNS: noun, plural
PDT: predeterminer
POS: possessive ending
PRP: pronoun, personal
PRP$: pronoun, possessive
RB: adverb
RBR: adverb, comparative
RBS: adverb, superlative
RP: adverb, particle
SYM: symbol
TO: infinitival "to"
UH: interjection
VB: verb, base form
VBD: verb, past tense
VBG: verb, gerund

In [ ]:
import utils

t = "I ate food grasiously. He has eaten. She eats. They eat\nThere was terror. Terrorism is bad. There were terrorists."
lemmas = utils.get(text)
print(lemmas)
for l in lemmas:
    print(f"{l.lemma_}, {l.pos_}, {l.is_stop}, {l.is_punct}")

German Chancellor Angela Merkel died in 1936 in New York. She got shot by a mysterious terrorist, terror, terrorism
German, PROPN, False, False
Chancellor, PROPN, False, False
Angela Merkel, PROPN, False, False
die, VERB, False, False
in, ADP, True, False
1936, NUM, False, False
in, ADP, True, False
New York, PROPN, False, False
., PUNCT, False, True
she, PRON, True, False
get, VERB, False, False
shoot, VERB, False, False
by, ADP, True, False
a, DET, True, False
mysterious, ADJ, False, False
terrorist, NOUN, False, False
,, PUNCT, False, True
terror, NOUN, False, False
,, PUNCT, False, True
terrorism, NOUN, False, False
